In [142]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction import DictVectorizer
from sklearn import preprocessing, model_selection

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
import xml.etree.cElementTree as ET
import gensim

from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk import pos_tag
import string
from gensim.models.word2vec import Word2Vec
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, f1_score
import gensim.downloader as api
import spacy

In [143]:
path = './data/restaurants_laptop_train_with_pos.csv'
df = pd.read_csv(path)
punc = []
for p in string.punctuation:
    punc.append(p)
df["text"] = df.apply(lambda r: r["text"].lower(), axis = 1)
df = df[~df.text.isin(punc)]

In [144]:
dataset = api.load("text8")  # load dataset as iterable
model = Word2Vec(dataset)
#model = Word2Vec(sentences=common_texts, vector_size=100, window=5, min_count=1, workers=4)
model.save("word2vec.model")
model = Word2Vec.load("word2vec.model")

In [145]:
train, test = train_test_split(df, test_size = 0.2, random_state=42)
le = preprocessing.LabelEncoder()

y_test= test.aspect_tag.values
y_test = le.fit_transform(y_test)
y_train = train.aspect_tag.values
y_train = le.fit_transform(y_train)
train["word2vec"] = train.apply(lambda r: model.wv[r["text"]] if r["text"] in model.wv else model.wv["default"], axis = 1) 
test["word2vec"] = test.apply(lambda r: model.wv[r["text"]] if r["text"] in model.wv else model.wv["default"], axis = 1)

C:\Users\sr_te\AppData\Local\Temp/ipykernel_15936/2943963045.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["word2vec"] = train.apply(lambda r: model.wv[r["text"]] if r["text"] in model.wv else model.wv["default"], axis = 1)
C:\Users\sr_te\AppData\Local\Temp/ipykernel_15936/2943963045.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["word2vec"] = test.apply(lambda r: model.wv[r["text"]] if r["text"] in model.wv else model.wv["default"], axis = 1)


In [146]:
X_train = train.word2vec.to_list()
X_test = test.word2vec.to_list()

In [147]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

LogisticRegression()

In [148]:
pred = lr.predict(X_test)
pred, y_test

(array([1, 1, 1, ..., 1, 1, 1]), array([1, 1, 1, ..., 1, 1, 1]))

In [141]:
print('Testing accuracy: {:.2f}%'.format(lr.score(X_test, y_test)*100))

Testing accuracy: 90.63%
